In [1]:
"""
notebook designed to create radar fence diagrams animations

### psuedocode ###
1. read in list of radar profiles
2. get full path to profiles
3. ingest and pull proc array, lon, lat, elev
4. convert x,y,z to utm coords
5. use pyvista to display profiles in 3d

BST 09DEC2021
python3
"""
### impots ###
import sys, os, itertools, glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import plotly.graph_objects as go
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point, LineString
from scipy.interpolate import griddata
import pyvista as pv
import scipy
import utm
# from mayavi import mlab

# add RAGU to path to import radar data
sys.path.append("C:/Users/btober/OneDrive - University of Arizona/Documents/code/radar/ragu/code")
from ingest import ingest
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 8]

In [2]:
# set up a method for reading radar data into pyvista format for plotting
def rdr2pv(rdata, fac=50, sampclip=[0,-1], ve=1, srf_samp_array=None, bed_samp_array=None, ws=1.68e8, zshift=None):
    """
    input:
    rdata = ragu garlic data object
    fac = int; along-track sampling factor - take every nth trace
    sampclip = list; [top sample #, bottom sample #], pull this horizontal slice of data array
    ve = int; vertical exaggeration factor
    srf_samp_array = array; optional array of len rdata.tnum containing the along-profile sample of the surface
    bed_samp_array = array; optional array of len rdata.tnum containing the along-profile sample of the bed
    ws = float; wavespeed for which to calculate basal elevation
    zshift = float; optional vertical shift of profile if profiles are not aligning
    
    
    return: 
    pv_data_dict = dictionary object containing pyvista bundled data objects for easy plotting
        pv_data_dict["surface"] = pyvista StructuredGrid object containing data profile in 3D space
        pv_data_dict["srf_collar"] = pyvista PolyData object containing surface elevation path in 3D space
        pv_data_dict["bed_collar"] = pyvista PolyData object containing basal elevation path in 3D space
        pv_data_dict["clim"] = list; [vmin, vmax] minimum and maximum color scale limits for plotting
    """

    # first create dict to hold all pyvista formatted data
    pv_data_dict = {}
    # downsample
    data = rdata.proc.get_curr_dB()
    data = data[sampclip[0]:sampclip[1],::fac]
    s = data.shape
    nsamps = s[0]
    ntraces = s[1]
    x, y, _, _ = utm.from_latlon(rdata.navdf['lat'].values, rdata.navdf['lon'].values)
    # x=rdata0.navdf.lon.to_numpy()
    # y=rdata0.navdf.lat.to_numpy()
    x = x[::fac]
    y = y[::fac]
    z = rdata.navdf['elev'].astype(float).values
    z = z[::fac]
    if zshift:
        z = z+zshift
    path = pv.PolyData(np.c_[x,y,z])
    
    # convert pick horizons to pyvista path
    if srf_samp_array is not None:
        srf = z- (3e8*ve*rdata.dt*srf_samp_array[::fac]/2)
        collar_srf = pv.PolyData(np.c_[x,y,srf])
        pv_data_dict["srf_collar"] = collar_srf
        if bed_samp_array is not None:
            bed = srf - ((bed_samp_array[::fac] - srf_samp_array[::fac])*rdata.dt*ve*ws/2)
            collar_bed = pv.PolyData(np.c_[x,y,bed])
            pv_data_dict["bed_collar"] = collar_bed
            
    # Create the depth vector and scale it
    z_range = np.arange(nsamps) * ve
    # repeat the Z locations across
    tp = np.repeat(z_range.reshape((-1, nsamps)), ntraces, axis=0)
    tp = path.points[:,2][:,None] - tp
    points = np.repeat(path.points, nsamps, axis=0)
    points[:,-1] = tp.ravel()

    # Make a surface to place the radar data on
    surface = pv.StructuredGrid()
    surface.points = points
    surface.dimensions = (1, nsamps, ntraces)
    # Associate the radar data to that surface - fortran raveling
    surface.point_arrays['dB'] = data.ravel(order='F')
    pv_data_dict["surface"] = surface
    
    pv_data_dict["clim"] = [np.floor(np.nanpercentile(rdata.proc.get_curr_dB(),5)),
                            np.ceil(np.nanpercentile(rdata0.proc.get_curr_dB(),100))]
    
    return pv_data_dict

In [3]:
# ingest profiles
flist = ['C:/Users/btober/OneDrive - University of Arizona/Documents/data/radar/testdata/ares/IRARES1B_20180818-000402.h5',
         'C:/Users/btober/OneDrive - University of Arizona/Documents/data/radar/testdata/ares/IRARES1B_20190928-235534.h5']


# ingest
igst0 = ingest(flist[0])
rdata0 = igst0.read("", "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs", "earth")

# ingest
igst1 = ingest(flist[1])
rdata1 = igst1.read("", "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs", "earth")

----------------------------------------
Loading: IRARES1B_20180818-000402
----------------------------------------
Loading: IRARES1B_20190928-235534


In [4]:
# import picks
pk_files = ["C:/Users/btober/OneDrive - University of Arizona/Documents/data/radar/IRARES1B/pick/IRARES1B_20180818-000402_pk_bst.csv",
            "C:/Users/btober/OneDrive - University of Arizona/Documents/data/radar/IRARES1B/pick/IRARES1B_20190928-235534_pk_bst.csv"]

igst0.import_pick(pk_files[0], "bst")
srf0 = rdata0.pick.horizons["srf"]
bed0 = rdata0.pick.horizons["bed"]
# handle ragu ingest legacy bug
if np.all(np.isnan(bed0)):
    bed0 = rdata0.pick.horizons["bed_imported"]
    
igst1.import_pick(pk_files[1], "bst")
srf1 = rdata1.pick.horizons["srf"]
bed1 = rdata1.pick.horizons["bed"]
# handle ragu ingest legacy bug
if np.all(np.isnan(bed1)):
    bed1 = rdata1.pick.horizons["bed_imported"]

In [5]:
# generate pyvista data objects for plotting data
pv_data_dict0 = rdr2pv(rdata0, fac=50, sampclip=[0,2000], ve=10, srf_samp_array=srf0, bed_samp_array=bed0, ws=1.68e8, zshift=800)
pv_data_dict1 = rdr2pv(rdata1, fac=50, sampclip=[0,2000], ve=10, srf_samp_array=srf1, bed_samp_array=bed1, ws=1.68e8)


C:\Users\btober\anaconda3\envs\py3\lib\site-packages\pyvista\core\dataset.py:1192: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(


In [6]:
# Load up pyvista objects in interactive plot
p = pv.Plotter(shape=(2,1),notebook=False,lighting='none')
labels = dict(xlabel="easting (m)", ylabel="northing (m)", zlabel="elevation (m.a.s.l)")

# subplot 0
p.subplot(0,0)
# add surface meshes
p.add_mesh(pv_data_dict0["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict0["clim"], show_scalar_bar=False)
p.add_mesh(pv_data_dict1["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict1["clim"], show_scalar_bar=False)
p.add_axes(**labels)
# p.show_grid(**labels)


# subplot 1
p.subplot(1,0)
# add surface meshes
p.add_mesh(pv_data_dict0["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict0["clim"], show_scalar_bar=False)
p.add_mesh(pv_data_dict1["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict1["clim"], show_scalar_bar=False)
# add interpretation paths
p.add_mesh(pv_data_dict0["srf_collar"], smooth_shading=True, color='tab:blue')
p.add_mesh(pv_data_dict0["bed_collar"], smooth_shading=True, color='tab:orange')
p.add_mesh(pv_data_dict1["srf_collar"], smooth_shading=True,color='tab:blue')
p.add_mesh(pv_data_dict1["bed_collar"], smooth_shading=True,color='tab:orange')
p.add_axes(**labels)
# p.show_grid(**labels)


# link all the views
p.link_views()  
p.show()

In [7]:
# create animation
p = pv.Plotter(shape=(2,1),window_size=([1920, 2160]),off_screen=True,notebook=False,lighting='none')
p.camera_position = 'yz'
p.camera.focal_point = (0.0, 0.0, .75)
p.camera.elevation = 30
labels = dict(xlabel="easting (m)", ylabel="northing (m)", zlabel="elevation (m.a.s.l)")
viewup = [0, 0,1]


# subplot 0
p.subplot(0,0)
# add surface meshes
p.add_mesh(pv_data_dict0["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict0["clim"], show_scalar_bar=False)
p.add_mesh(pv_data_dict1["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict1["clim"], show_scalar_bar=False)
p.add_axes(**labels)
# zoom in
p.camera.zoom(2.125)
# p.show_grid(**labels)


# subplot 1
p.subplot(1,0)
# add surface meshes
p.add_mesh(pv_data_dict0["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict0["clim"], show_scalar_bar=False)
p.add_mesh(pv_data_dict1["surface"], smooth_shading=True,scalars='dB', opacity = .999,cmap='Greys_r',clim=pv_data_dict1["clim"], show_scalar_bar=False)
# add interpretation paths
p.add_mesh(pv_data_dict0["srf_collar"], smooth_shading=True, color='tab:blue')
p.add_mesh(pv_data_dict0["bed_collar"], smooth_shading=True, color='tab:orange')
p.add_mesh(pv_data_dict1["srf_collar"], smooth_shading=True,color='tab:blue')
p.add_mesh(pv_data_dict1["bed_collar"], smooth_shading=True,color='tab:orange')
p.add_axes(**labels)
# zoom in
p.camera.zoom(2.125)
# p.show_grid(**labels)


# link all the views
p.link_views()  
p.show(auto_close=False,return_viewer=False)
# generate path to create image frames and orbit the plot
path = p.generate_orbital_path(n_points=225, shift=pv_data_dict0["surface"].length)
p.open_movie("C:/Users/btober/OneDrive - University of Arizona/Documents/pres/AGU2021/malaspina_orbit.mp4")
p.orbit_on_path(path, viewup=viewup,write_frames=True)
p.close()